In [239]:
import pandas as pd 
import numpy as np 
import tensorly as tl
import plotly.express as px
import plotly.io as pio
import os 
pio.renderers.default = "notebook_connected"

In [240]:
def read_ticker(ticker) : 
    filename = ticker
    csv = pd.read_csv(filename)
    return csv 


# def calc_momentum(df):

    

In [349]:
tickers = ['BAC','KO','UNH','BRK.B','NVDA','V', 'DIS','PFE']
# tickers = ['AAPL']
df_tickers = []
N = 12 # group by 12 rows
YEARS = np.arange(2021,2009,-1)
print(YEARS)
PATH = os.getcwd()
for ticker in tickers : 
    path = os.path.join(PATH,'EQUITIES', ticker+'.xlsx')
    df = pd.read_excel(path,skiprows=6)
    df = df[:-1]  #skip last row 
    df.drop([0],inplace=True) #delete the first row
    df_tickers.append(df)


bond_names = ['us_3_month','us_6_month','us_12_month','us_2_year','us_5_year','us_10_year','us_30_year']


df_bonds = []
for bond in bond_names : 
    path = os.path.join(PATH,'BONDS', bond+'.xlsx')
    df = pd.read_excel(path,skiprows=5)
    df = df[:-1]
    df.drop([0],inplace=True)
    df_bonds.append(df)



[2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010]


## Add momentum to equities and plot it 


In [359]:
def generate_momentum_data(list_df, list_names) : 
    yearly_price_df = pd.DataFrame()
    logged_yearly_price_df = pd.DataFrame()
    momentum_df = pd.DataFrame()
    for df,ticker in zip(list_df,list_names) : 
        df = df.reset_index()
        yearly_price = df.groupby(df.index // N).mean()['PX_LAST']
        print(ticker)
        yearly_price.index = YEARS
        logged_yearly_price_df[ticker] = np.log(yearly_price) 
        yearly_price_df[ticker] = yearly_price
        momentum_df[ticker] = ((yearly_price - yearly_price.shift(-1)) / yearly_price.shift(-1)) * 100
    return momentum_df, logged_yearly_price_df, yearly_price_df

def generate_momentum_plots(momentum_df, yearly_price_df, str_asset) : 
    # fig = px.line(yearly_price_df)

    # fig.update_layout(
    #     title = f'Yearly Price of {str_asset} from 2010-2021', # adding the title 
    #     xaxis_title = 'Dates', # title for x axis 
    #     yaxis_title = 'Yearly Log Price', # title for y axis 
    #     xaxis = dict(           # attribures for x axis 
    #         showline = True,
    #         showgrid = True,
    #         linecolor = 'black',

    #     ),
    #     yaxis = dict(           # attribures for y axis 
    #         showline = True,
    #         showgrid = True,
    #         linecolor = 'black',
    #     ),
    # )

    # fig.show()



    # fig = px.histogram(momentum_df,barmode='group')

    # fig.update_layout(
    #     title = 'Momentum', # adding the title 
    #     xaxis_title = 'Momentum (%)', # title for x axis 
    #     yaxis_title = 'Number of occurences', # title for y axis 
    #     xaxis = dict(           # attribures for x axis 
    #         showline = True,
    #         showgrid = True,
    #         linecolor = 'black',
    #         range = [-50, 160]

    #     ),
    #     yaxis = dict(           # attribures for y axis 
    #         showline = True,
    #         showgrid = True,
    #         linecolor = 'black',
    #     ),
    # )

    # fig.show()


    bins = [-300,0,300]
    binned_mom_df = pd.DataFrame()
    for column in momentum_df.columns :
        str = column
        binned_mom_df[str] = pd.cut(momentum_df[column],bins,labels = ['Negative', 'Positive'])

    print(binned_mom_df)
    fig = px.histogram(binned_mom_df,barmode = 'group')

    fig.update_xaxes(categoryorder='array', categoryarray= ['Negative', 'Positive'])

    fig.update_layout(
        title = 'Positive or Negative Momentum throughout 2010-2021', # adding the title 
        xaxis_title = 'Positive or Negative', # title for x axis 
        yaxis_title = 'Number of occurences', # title for y axis 
        xaxis = dict(           # attribures for x axis 
            showline = True,
            showgrid = True,
            linecolor = 'black',

        ),
        yaxis = dict(           # attribures for y axis 
            showline = True,
            showgrid = True,
            linecolor = 'black',
        ),
    )
    fig.show()


momentum_eq_df, logged_yearly_price_eq_df, yearly_price_eq_df = generate_momentum_data(df_tickers,tickers)

generate_momentum_plots(momentum_eq_df, yearly_price_eq_df, str_asset='Equities')

momentum_bonds_df, logged_yearly_price_bonds_df, yearly_price_bonds_df = generate_momentum_data(df_bonds,bond_names)

generate_momentum_plots(momentum_bonds_df, yearly_price_bonds_df, str_asset='Bonds')






BAC
KO
UNH
BRK.B
NVDA
V
DIS
PFE
           BAC        KO       UNH     BRK.B      NVDA         V       DIS  \
2021  Positive  Positive  Positive  Positive  Positive  Positive  Positive   
2020  Negative  Negative  Positive  Negative  Positive  Positive  Negative   
2019  Positive  Positive  Positive  Positive  Negative  Positive  Positive   
2018  Positive  Positive  Positive  Positive  Positive  Positive  Positive   
2017  Positive  Positive  Positive  Positive  Positive  Positive  Positive   
2016  Negative  Positive  Positive  Positive  Positive  Positive  Negative   
2015  Positive  Positive  Positive  Positive  Positive  Positive  Positive   
2014  Positive  Positive  Positive  Positive  Positive  Positive  Positive   
2013  Positive  Positive  Positive  Positive  Positive  Positive  Positive   
2012  Negative  Positive  Positive  Positive  Negative  Positive  Positive   
2011  Negative  Positive  Positive  Negative  Positive  Positive  Positive   
2010       NaN       NaN       N

us_3_month
us_6_month
us_12_month
us_2_year
us_5_year
us_10_year
us_30_year
     us_3_month us_6_month us_12_month us_2_year us_5_year us_10_year  \
2021   Negative   Negative    Negative  Negative  Positive   Positive   
2020   Negative   Negative    Negative  Negative  Negative   Negative   
2019   Positive   Negative    Negative  Negative  Negative   Negative   
2018   Positive   Positive    Positive  Positive  Positive   Positive   
2017   Positive   Positive    Positive  Positive  Positive   Positive   
2016        NaN   Positive    Positive  Positive  Negative   Negative   
2015   Positive   Positive    Positive  Positive  Negative   Negative   
2014   Negative   Negative    Negative  Positive  Positive   Positive   
2013   Negative   Negative    Negative  Positive  Positive   Positive   
2012   Positive   Positive    Negative  Negative  Negative   Negative   
2011   Negative   Negative    Negative  Negative  Negative   Negative   
2010        NaN        NaN         NaN       NaN

## Plot the charts


In [ ]:
import plotly.express as px 



In [244]:
from tensorly.decomposition import tucker, parafac, non_negative_tucker

momentum_df = (momentum_df).iloc[:-1] #Remove NaN rows
momentum_df

,AAPL,MSFT,AMZN,BRKB,NVDA,JNJ
2021,75.688914,42.824307,53.596000,-3.848265,94.015528,5.895909
2020,20.159329,34.203929,8.291905,4.013517,132.167276,3.823708
2019,20.527346,35.896927,61.889552,12.100223,-16.304122,-0.166823
2018,45.800794,33.596029,42.738199,22.254711,34.902303,14.847634
2017,-9.801002,17.817753,41.796979,7.497805,157.204685,15.126830
2016,20.946833,11.041627,56.800714,2.668773,156.868674,-2.603205
2015,43.277594,27.727659,4.862980,19.923195,30.499488,17.388522
2014,-18.131593,14.388181,33.978356,31.323567,29.961765,28.248352
2013,54.148372,12.572411,17.446598,8.616527,9.960395,5.310283
2012,38.968874,-0.983915,38.559124,-0.963848,-18.709820,5.622479


In [307]:



# print(np.shape(momentum_tensor))

def parafac_decomp(df, r) : 

    '''
    Input : 
    df : dataframe containing a 2D tensor ready for decomposition

    r : Value of r for the r-rank approximation




    Return  :
    reconstructed_df : Dataframe of the r-rank approximation of the tensor
    error : L1-norm error 
    
    '''
    tensor = df.to_numpy()
    weights, factors = parafac(tensor, rank=r)
    print(f"Rank = {rank}")
    print(f"Number of factors {len(factors)}")
    
    outer_prod= np.zeros(shape = np.shape(tensor))

    A = factors[0]
    B = factors[1]
    individual_ranks = []
    for i in range(rank) :
        a = A[:,i] 
        b = B[:,i]
        outer_prod += np.outer(a,b)
        individual_ranks.append(pd.DataFrame(np.outer(a,b),index=momentum_df.index,columns=momentum_df.columns))
    reconstructed_df = pd.DataFrame(outer_prod,index=momentum_df.index,columns=momentum_df.columns)
    error = np.abs((reconstructed_df - momentum_df).values.sum())
    return individual_ranks, reconstructed_df, error

rank = 6 
individual_ranks, r_rank_approxed, error = parafac_decomp(momentum_df,rank)


Rank = 6
Number of factors 2
           AAPL       MSFT       AMZN       BRKB        NVDA        JNJ
2021  75.688914  42.824307  53.596000  -3.848265   94.015528   5.895909
2020  20.159329  34.203929   8.291905   4.013517  132.167276   3.823708
2019  20.527346  35.896927  61.889552  12.100223  -16.304122  -0.166823
2018  45.800794  33.596029  42.738199  22.254711   34.902303  14.847634
2017  -9.801002  17.817753  41.796979   7.497805  157.204685  15.126830
2016  20.946833  11.041627  56.800714   2.668773  156.868674  -2.603205
2015  43.277594  27.727659   4.862980  19.923195   30.499488  17.388522
2014 -18.131593  14.388181  33.978356  31.323567   29.961765  28.248352
2013  54.148372  12.572411  17.446598   8.616527    9.960395   5.310283
2012  38.968874  -0.983915  38.559124  -0.963848  -18.709820   5.622479
2011  41.769937  -5.776851  14.028653   3.506477   16.684902  -2.715028
           AAPL       MSFT       AMZN       BRKB        NVDA        JNJ
2021  29.037688  25.947016  40.4464